In [4]:
from google.cloud import storage
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import cm
from datetime import datetime
import glob
import os
from io import StringIO
import json
import pickle
import six
import charset_normalizer
from wordcloud import WordCloud 

sns.set()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

## init

In [5]:
storage_client =  storage.Client.from_service_account_json('./heidless-jupyter-0-d2008100d98c.json')

BUCKET_NAME = 'heidless-jupyter-bucket-0'

bucket = storage_client.get_bucket(BUCKET_NAME)

## read files (data)

In [6]:

#AllCSV = []
#my_prefix = 'developer-survey/2019/'
#my_file = 'survey_results_public.csv'
#full_file = my_prefix + my_file
##print(f'full_file: {full_file}')#

#file_names = list(bucket.list_blobs(prefix=my_prefix))
#for file in file_names:
#    if(file.name != my_prefix):
#        if file.name == full_file:
#            AllCSV.append(file.name)
#            print(file.name)
#AllCSV

## build dataframe

In [8]:

#all_dataframes = []
my_prefix = 'developer-survey/2019/'
my_file = 'survey_results_public.csv'
full_file = my_prefix + my_file

blob = bucket.get_blob(full_file)
if blob is not None and blob.exists(storage_client):
    bt = blob.download_as_string()
    s = str(bt, 'ISO-8859-1')
    s = StringIO(s)
    print(full_file)
    df = pd.read_csv(s, encoding='ISO-8859-1', low_memory=False)

developer-survey/2019/survey_results_public.csv
developer-survey/2019/survey_results_public.csv


In [9]:
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or ...",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a jo...","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,Taken an online course in programming or softw...,NaN,"Developer, desktop or enterprise applications;...",NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,I am actively looking for a job,I've never had a job,NaN,NaN,Financial performance or funding status of the...,"Something else changed (education, award, medi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,NaN,MySQL,Windows,Windows,Django,Django,NaN,NaN,Atom;PyCharm,Windows,I do not use containers,NaN,Useful across many domains and could change ma...,Yes,Yes,Yes,Instagram,Online,Username,2017,Daily or almost daily,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,11-30 minutes,Yes,A few times per month or weekly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelorâs degree (BA, BS, B.Eng., etc.)",Web development or web design,"Taught yourself a new language, framework, or ...",100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,22,1,Slightly satisfied,Slightly satisfied,Not at all confident,Not sure,Not sure,"Iâm not actively looking, but I am open to n...",1-2 years ago,Interview with people in peer roles,No,"Languages, frameworks, and other techno

## read file (schema)

In [11]:
my_prefix = 'developer-survey/2019/'
my_file = 'survey_results_schema.csv'
full_file = my_prefix + my_file

blob = bucket.get_blob(full_file)
if blob is not None and blob.exists(storage_client):
    bt = blob.download_as_string()
    s = str(bt, 'ISO-8859-1')
    s = StringIO(s)
    print(full_file)
    schema_df = pd.read_csv(s, encoding='ISO-8859-1', low_memory=False)
    
schema_df

developer-survey/2019/survey_results_schema.csv


,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,OpenSourcer,How often do you contribute to open source?
4,OpenSource,How do you feel about the quality of open sour...
5,Employment,Which of the following best describes your cur...
6,Country,In which country do you currently reside?
7,Student,"Are you currently enrolled in a formal, degree..."
8,EdLevel,Which of the following best describes the high...
9,UndergradMajor,What was your main or most important field of ...


In [19]:
schema_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Column        85 non-null     object
 1   QuestionText  85 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [20]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [21]:
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
62723,63129,I am a developer by profession,Yes,Never,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Afghanistan,"Yes, full-time",I never completed any formal education,NaN,Taken an online course in programming or softw...,"Just me - I am a freelancer, sole proprietor, ...",NaN,Less than 1 year,Younger than 5 years,1,Very dissatisfied,Very dissatisfied,Very confident,Yes,Yes,I am actively looking for a job,I've never had a job,Write any code,Yes,How widely used or impactful my work output wo...,I had a negative experience or interaction at ...,AED,United Arab Emirates dirham,648838511.0,Monthly,1000000.0,168.0,There is a schedule and/or spec (made by me or...,Being tasked with non-development work,It's complicated,"Other place, such as a coworking space or cafe",Far below average,No,NaN,NaN,NaN,NaN,Assembly;Bash/Shell/PowerShell;Other(s):,Bash/Shell/PowerShell;TypeScript,MySQL,MySQL,Android;iOS;Windows;WordPress;Other(s):,Android;iOS;Windows;WordPress;Other(s):,Angular/Angular.js;Express;jQuery;Laravel;Othe...,Angular/Angular.js;Express;jQuery;Laravel;Othe...,Ansible;Apache Spark;Torch/PyTorch,Apache Spark;Torch/PyTorch,Android Studio;Atom;Coda;Eclipse;Emacs;IPython...,Windows,"Outside of work, for personal projects",Accepting Bitcoin or other coins and tokens as...,Useful across many domains and could change ma...,Yes,Yes,Yes,I don't use social media,Online,UserID,NaN,I have never visited Stack Overflow (before to...,Find answers to specific questions,Less than once per week,Stack Overflow was much faster,0-10 minutes,NaN,NaN,"No, I didn't know that Stack Overflow had a jo...",NaN,"No, not at all",NaN,Tech articles written by other developers,NaN,NaN,NaN,NaN,NaN,Yes,Too short,Easy
50172,50499,"I am not primarily a developer, but I write co...",Yes,Less than once per year,"OSS is, on average, of LOWER quality than prop...",Employed full-time,Afghanistan,No,"Bachelorâs degree (BA, BS, B.Eng., etc.)","A health science (ex. nursing, pharmacy, radio...",Participated in a full-time developer training...,"10,000 or more employees",Data or business analyst;Database administrato...,7,27,7,Very dissatisfied,Very dissatisfied,Somewhat confident,Yes,Yes,"Iâm not actively looking, but I am open to n...",More than 4 years ago,"Write code by hand (e.g., on a whiteboard);Int...",No,Opportunities for professional development;Div...,"Something else changed (education, award, medi...",AFN,Afghan afghani,960000.0,Monthly,153216.0,40.0,There is a schedule and/or spec (made by me or...,Lack of support from management;Time spent com...,Less than once per month / Never,Office,A little above average,"Yes, because I see value in code review",8.0,"Yes, it's not part of our process but the deve...",Developers typically have the most influence o...,I have a great deal of influence,C#;HTML/CSS;JavaScript;SQL;VBA,C#;HTML/CSS;JavaScript;R;SQL;TypeScript;VBA,Microsoft SQL Server;PostgreSQL;SQLite,MariaDB;MongoDB;Microsoft SQL Server;PostgreSQ...,NaN,Android,ASP.NE

In [32]:
df.sort_values(by=['Country', 'ConvertedComp'], ascending=[True,False], inplace=True)

In [33]:
df[['Country', 'ConvertedComp']].head(50)

,Country,ConvertedComp
62723,Afghanistan,1000000.0
50172,Afghanistan,153216.0
39018,Afghanistan,19152.0
58082,Afghanistan,17556.0
7056,Afghanistan,14364.0
22327,Afghanistan,7980.0
48122,Afghanistan,4464.0
10697,Afghanistan,3996.0
8112,Afghanistan,1596.0
29560,Afghanistan,1116.0


In [34]:
df['ConvertedComp'].nlargest(10)

25833    2000000.0
87353    2000000.0
21895    2000000.0
28080    2000000.0
72274    2000000.0
77665    2000000.0
79701    2000000.0
51798    2000000.0
75088    2000000.0
32056    2000000.0
Name: ConvertedComp, dtype: float64

In [35]:
df.nlargest(10, 'ConvertedComp')

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
25833,25983,I am a developer by profession,Yes,Less than once per year,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Canada,No,"Bachelorâs degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Received on-the-job training in software devel...,"10,000 or more employees","Developer, full-stack",8,16,2,Very satisfied,Very satisfied,Somewhat confident,No,No,"Iâm not actively looking, but I am open to n...",3-4 years ago,Solve a brain-teaser style puzzle;Interview wi...,No,Remote work options;Opportunities for professi...,"My job status changed (promotion, new job, etc.)",USD,United States dollar,65500.0,Weekly,2000000.0,45.0,There is a schedule and/or spec (made by me or...,Being tasked with non-development work;Meeting...,A few days each month,Office,Average,"Yes, because I see value in code review",2.0,"Yes, it's part of our process",Developers and management have nearly equal in...,I have little or no influence,Bash/Shell/PowerShell;C++;C#;HTML/CSS;JavaScri...,Bash/Shell/PowerShell;C#;F#;Python;SQL;TypeScript,Microsoft SQL Server;MySQL,Microsoft SQL Server;MySQL,AWS;Docker;Heroku;Linux;MacOS;Raspberry Pi;Win...,AWS;Docker;Linux;MacOS;Windows,Angular/Angular.js,Angular/Angular.js;Flask;React.js,.NET;.NET Core;Node.js,.NET Core;Node.js,Notepad++;Sublime Text;Vim;Visual Studio;Visua...,Windows,Development;Production,NaN,"Useful for decentralized currency (i.e., Bitcoin)",No,Yes,Yes,Reddit,In real life (in person),Username,2013,Daily or almost daily,Find answers to specific questions;Learn how t...,1-2 times per week,Stack Overflow was much faster,11-30 minutes,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","No, not really",Just as welcome now as I felt last year,NaN,24.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
87353,87896,I am a developer by profession,Yes,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,Germany,No,"Bachelorâs degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Taken an online course in programming or softw...,2-9 employees,"Developer, full-stack",9,17,8,Very satisfied,Very satisfied,Somewhat confident,No,I am already a manager,I am not interested in new job opportunities,More than 4 years ago,Write any code;Interview with people in senior...,No,Office environment or company culture;Diversit...,"Something else changed (education, award, medi...",USD,United States dollar,55000.0,Weekly,2000000.0,40.0,There is a schedule and/or spec (made by me or...,Being tasked with non-development work;Meeting...,All or almost all the time (I'm full-time remote),Home,A little above average,"Yes, because I see value in code review",3.0,"No, but I think we should",Developers and management have nearly equal in...,I have a great deal of influence,HTML/CSS;JavaScript;PHP;SQL,HTML/CSS;JavaScript;PHP;Ruby

In [22]:
df.sort_values(by=['Country', 'ConvertedComp'], ascending=[True, False], inplace=True)

In [23]:
df[['Country', 'ConvertedComp']].head(50)

,Country,ConvertedComp
62723,Afghanistan,1000000.0
50172,Afghanistan,153216.0
39018,Afghanistan,19152.0
58082,Afghanistan,17556.0
7056,Afghanistan,14364.0
22327,Afghanistan,7980.0
48122,Afghanistan,4464.0
10697,Afghanistan,3996.0
8112,Afghanistan,1596.0
29560,Afghanistan,1116.0


In [17]:
df['ConvertedComp'].nlargest(10)

25833    2000000.0
87353    2000000.0
21895    2000000.0
28080    2000000.0
72274    2000000.0
77665    2000000.0
79701    2000000.0
51798    2000000.0
75088    2000000.0
32056    2000000.0
Name: ConvertedComp, dtype: float64

In [18]:
df.nsmallest(10, 'ConvertedComp')

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
719,722,I am a developer by profession,No,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,NaN,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Less than 1 year,Younger than 5 years,Less than 1 year,Very satisfied,Very satisfied,NaN,NaN,NaN,I am not interested in new job opportunities,NA - I am an independent contractor or self em...,NaN,No,NaN,I heard about a job opportunity (from a recrui...,AFN,Afghan afghani,1.0,Yearly,0.0,1.0,NaN,NaN,Less than once per month / Never,"Other place, such as a coworking space or cafe",Far above average,"Yes, because I see value in code review",1.0,"No, and I'm glad we don't",NaN,I have little or no influence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,SIGH,What?,NaN,Neither,NaN,2008,Multiple times per day,NaN,More than 10 times per week,The other resource was much faster,60+ minutes,Not sure / can't remember,NaN,Yes,Yes,Not sure,Not applicable - I did not use Stack Overflow ...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Too long,Difficult
28470,28638,I am a developer by profession,Yes,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...","Independent contractor, freelancer, or self-em...",Afghanistan,NaN,"Other doctoral degree (Ph.D, Ed.D., etc.)",I never declared a major,NaN,NaN,NaN,36,13,16,Very dissatisfied,Very dissatisfied,NaN,NaN,NaN,I am not interested in new job opportunities,NaN,NaN,NaN,NaN,NaN,AFN,Afghan afghani,3.0,Monthly,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bash/Shell/PowerShell;C;C++;Java;Objective-C;O...,Bash/Shell/PowerShell;C;C++;Java;Objective-C;O...,MySQL;SQLite,MySQL;SQLite,Android;iOS;Linux;MacOS,Android;iOS;Linux,jQuery,NaN,TensorFlow;Unity 3D,Hadoop;TensorFlow;Unity 3D,Emacs,Linux-based,I do not use containers,NaN,Useful for immutable record keeping outside of...,No,Yes,No,NaN,In real life (in person),Username,2008,Daily or almost daily,Find answers to specific questions,6-10 times per week,The other resource was much faster,11-30 minutes,Yes,Less than once per month or monthly,Yes,"No, I've heard of them, but I am not part of a...","No, not at all",A lot less welcome now than last year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,NaN
13760,13825,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,NaN,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,13,83,Less than 1 year,Very dissatisfied,Very satisfied,NaN,NaN,NaN,I am actively looking for a job,NA - I am an independent contractor or self em...,Complete a take-home project;Solve a brain-tea...,Yes,Financial performance or funding status of the...,"Something else changed (education, award, medi...",AFN,Afghan afghani,1.0,Yearly,0.0,168.0,NaN,Being tasked with non-development work;Non-wor...,About half the time,Home,Far above average,No,NaN,"Yes, it's part of our process",NaN,I have a great deal of influence,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,Other(s):,NaN,NaN,I do not use containers,N